In [ ]:
#Our project is two steps:
#1. Detect if a given package has suspicious activity (a peak or a step in downloads)
#2. Determine whether that suspicious activity can be explained.

#Step 1 is doing the "magic math" and stuff on our pkg_list. (Already completed, but needs further validation)

#Step 2 has many parts. 
#2.1 Was the package added or dropped as a dependency on a popular project around the time of the rise/ fall of downloads? (Next step)
#2.2 Did any dependents have a similar rise/ fall in downloads? (In progress, but is proving very unreliable so may be dropped). 
#2.2 is probably as far as we will be able to make it, but we can discuss further steps in our presentation
#2.3 Did any developer/ nightly builds have our package as a dependency? 
#2.4 Did a new version release that correlates to increased downloads?
#Etc

In [ ]:
import gzip, json
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
    fc = f.read().decode("utf-8")
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            print("DECODE_STACKED() FAILED")
            pass
        yield obj

In [ ]:
#I don't think this is still needed
for obj in decode_stacked(fc):
    ind = 0
    try: versions = list(obj['versions'].keys())
    except: continue
    try: release_dates = obj['time']
    except: continue
    info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
    for key in versions:
        try:
            info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
            if pkg in info[key]['dependencies']: ind = 1
        except: continue
    if ind :
        dependent[pkg].append({obj['name']: info})

In [ ]:
for k in dependent.keys():
    if k == 'ajv-keywords':
        print(k, dependent[k][0])
        break

In [ ]:
#user-info
#    pji
#        version 1.0
#        date
#        
#        version 1.1
#        date
#    gifnoc
#        version
#        date
        

In [ ]:
#pkgs = ['htmlnano', 'object.map', 'babylon-walk', 'wix-style-react', 'threads', 'minimist-options', 'lead', '@babel/polyfill', '@babel/helper-define-map', 'cloudscraper']
pkgs = ['ajv-keywords']   
dependent = {}
for pkg in pkgs:
    dependent[pkg] = []
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent[pkg].append(obj['name'])

In [ ]:
for k in dependent.keys():
    for i in dependent[k]:
        print(i)
        #print(k, dependent[k][:20])
        #print("len = ", len(dependent[k]))
        #break

In [ ]:
#ajv-keywords
    #jsconscript-proxy
    #senasaikou-yeoman
    #webpack-egoist
#user-info
    #ajv-keywords
        

In [93]:
for k in dependent.keys():
    print(dependent[k])
    print("len = ", len(dependent[k]))
    break

['jsonscript-proxy', 'senasaikou-yeoman', 'webpack-egoist', 'alex-d.js', 'advanced-image-loader', 'webpack-addons-ylvis', 'primeng-custom', 'jicli', '@lgeiger/electron-builder', '@touched/map-tool-project', 'generator-arc', 'node-private-tools', 'fhir2', 'webpack-ie8', 'mocoolka-tools', 'misstep', '@cysonius/schema-utils', 'ng2-dnd-kokatsuna', 'qweb-client', '@johnf/electron-builder', 'qor', 'json-framework', 'hy-checkbox', 'babel-plugin-react-native-css', 'webpack-universal', 'babel-plugin-react-css-modules-avion', 'general-req', '@ngxvoice/ngx-voicelistner', 'electron-builder-admin', 'iqm', 'freemamba', 'isomorphic-webpack', 'webpack-tal', 'search-list-react', 'sea-floor', 'carousel-react', 'nicholas_util', 'search-input-react', 'json-injector', 'es-webpack', '@frctl/support', 'ajv-interface-keywords', 'stygian', 'lrbceshi', 'gh-lint', '@jose_santacruz/middy', '@ddder/webpack', 'vbot', 'outils-ren', 'canvas-fingerprint', 'firebase-forum', 'firebase-feedback', 'egeria-mercurius', 'web

In [79]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "pkg_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=6)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for pkg in pkgs:
    dl = []
    for r in dt_range:
        url = base_url+r[0]+':'+r[1]+'/'+pkg
        try: r = requests.get(url)
        except: print('Timeout '+ pkg+'\n')
        
        try:
            result = r.content
            try:
                rj = json.loads(result.decode('utf-8', errors='ignore'))
                rj.pop('package', None)
                dl = dl + [rj]
            except: print('DecodeError '+ pkg+'\n')
        except: print('BadURL '+ pkg+'\n')
    #ent = {pkg: dl}    
    ent = {'name':pkg, 'dl': dl}

    coll.insert_one(ent)
    print(ent)

1
{'dl': [{'end': '2015-03-08', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-15', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-22', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-29', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-05', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-12', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-19', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-26', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-03', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-10', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-17', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-24', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-31', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-07', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-14', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-21', 'downloads': 0, 'start': '2015-06-14'}

In [66]:
import math
total_list = []
start_date = []
end_date = []
download_list = []

for r in coll.find():
    download_list = []
    start_date = []
    ent = r['dl']
    for k in ent:
        download_list.append(k['downloads'])
        start_date.append(k["start"])
#map, reduce, filter, lambda
    log_list = [math.log(x+1) for x in download_list]
    difference_list = []
    for i in range(1,len(log_list)-1):
        dif = log_list[i] - log_list[i-1]
        difference_list.append( (log_list[i] - log_list[i-1]))
        if dif > 7:
            print("Start date of spike for " + r['name'] +": " +start_date[i])
        if dif < -7:
            print("Start date of fall for " + r['name'] +": " +start_date[i])
    #print(difference_list)
#print(log_list)
#print(download_list)


Start date of spike: 2017-07-12
Start date of spike: 2018-02-21
Start date of spike: 2018-02-05
Start date of fall: 2018-05-12


In [ ]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl2"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

#pkgs = ['ajv-keywords', 'htmlnano']
start_date = datetime.datetime(2015, 3, 1)
delta = datetime.timedelta(days=6)
end_date = start_date + delta
dt_range = []
final_date = datetime.datetime(2018, 10, 1)

while start_date < final_date:
    if end_date > final_date : end_date = final_date
    dt_range.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    start_date = end_date + datetime.timedelta(days=1)
    end_date = start_date + delta


# get data
base_url = 'https://api.npmjs.org/downloads/point/'
l = len(pkgs)
print(l)
for k in dependent.keys():
    for i in dependent[k]:
        dl = []
        for r in dt_range:
            url = base_url+r[0]+':'+r[1]+'/'+i
            try: r = requests.get(url)
            except: print('Timeout '+ i+'\n')
        
            try:
                result = r.content
                try:
                    rj = json.loads(result.decode('utf-8', errors='ignore'))
                    rj.pop('package', None)
                    dl = dl + [rj]
                except: print('DecodeError '+ i+'\n')
            except: print('BadURL '+ i+'\n')
        #ent = {pkg: dl}    
        ent = {'name':i, 'dl': dl}
        coll.insert_one(ent)
        print(ent)

1
{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

...

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

{'dl': [{'end': '2015-03-07', 'downloads': 0, 'start': '2015-03-01'}, {'end': '2015-03-14', 'downloads': 0, 'start': '2015-03-08'}, {'end': '2015-03-21', 'downloads': 0, 'start': '2015-03-15'}, {'end': '2015-03-28', 'downloads': 0, 'start': '2015-03-22'}, {'end': '2015-04-04', 'downloads': 0, 'start': '2015-03-29'}, {'end': '2015-04-11', 'downloads': 0, 'start': '2015-04-05'}, {'end': '2015-04-18', 'downloads': 0, 'start': '2015-04-12'}, {'end': '2015-04-25', 'downloads': 0, 'start': '2015-04-19'}, {'end': '2015-05-02', 'downloads': 0, 'start': '2015-04-26'}, {'end': '2015-05-09', 'downloads': 0, 'start': '2015-05-03'}, {'end': '2015-05-16', 'downloads': 0, 'start': '2015-05-10'}, {'end': '2015-05-23', 'downloads': 0, 'start': '2015-05-17'}, {'end': '2015-05-30', 'downloads': 0, 'start': '2015-05-24'}, {'end': '2015-06-06', 'downloads': 0, 'start': '2015-05-31'}, {'end': '2015-06-13', 'downloads': 0, 'start': '2015-06-07'}, {'end': '2015-06-20', 'downloads': 0, 'start': '2015-06-14'}, 

In [ ]:
import time, requests, sys, pymongo, json, datetime


dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]

for r in coll.find():
    temp = r['name']
    print(temp)

In [77]:
import math, time, requests, sys, pymongo, json, datetime

dbname = "NPMvulnerabilities"
collname = "dep_dl"
client = pymongo.MongoClient("da1.eecs.utk.edu")

db = client[dbname]
coll = db[collname]
#scaling up -- for pkg in pkgs?
print("For package " + pkg + ":")
for r in coll.find():
    download_list = []
    start_date = []
    ent = r['dl']
    flag = 0
    for k in ent:
        if k["downloads"] > 10000: #ADDITION--------------
            flag = 1
    if flag == 1:    
        for k in ent:
            download_list.append(k["downloads"])
            start_date.append(k["start"])
#map, reduce, filter, lambda
            
        log_list = [math.log(x+1) for x in download_list]
   # difference_list = []
    if flag == 1:
            for i in range(1,len(log_list)-1):
                dif = log_list[i] - log_list[i-1]
       # difference_list.append(dif)
                if dif > 3.5:
                    print("Start date of spike for " + r['name'] +": " +start_date[i])
                if dif < -2:
                    print("Start date of fall for " + r['name'] +": " +start_date[i])
                #if r['name'] == 'yoshi':
                 #   print(dif)


For package ajv-keywords:
Start date of spike for @webpack-contrib/schema-utils: 2018-04-02
Start date of spike for schema-utils: 2017-03-06
Start date of spike for schema-utils: 2017-05-01
Start date of spike for read-config-file: 2017-07-12
Start date of fall for yoshi: 2017-12-11
Start date of spike for yoshi: 2018-04-18
Start date of fall for yoshi: 2018-04-26
Start date of fall for yoshi: 2018-05-28
Start date of fall for yoshi: 2018-07-07


In [82]:
#revert to previous version and gets the old format of the versions. Gets dependent list with version history
pkgs = ['ajv-keywords']   
dependent1 = {}
for pkg in pkgs:
    dependent1[pkg] = []   
    for obj in decode_stacked(fc):
        ind = 0
        try: versions = list(obj['versions'].keys())
        except: continue
        try: release_dates = obj['time']
        except: continue
        info = {key: {'release_date': release_dates[key]} for key in release_dates if key not in ['created', 'modified']}
        for key in versions:
            try:
                info[key]['dependencies'] = list(obj['versions'][key]['dependencies'].keys()) list(obj['versions'][key]['dependencies'].keys())
                if pkg in info[key]['dependencies']: ind = 1
            except: continue
        if ind :
            dependent1[pkg].append({obj['name']: info})

In [87]:
for k in dependent1.keys():
    print(dependent1[k][1:2])
    print("len = ", len(dependent1[k]))
    break

[{'senasaikou-yeoman': {'1.1.0': {'dependencies': ['gulp-useref', 'browser-sync', 'gulp-eslint', 'babel-preset-es2015', 'gulp-sourcemaps', 'babel-core', 'babel-preset-react', 'gulp-htmlmin', 'gulp', 'wiredep', 'run-sequence', 'gulp-babel', 'gulp-cssnano', 'gulp-autoprefixer', 'gulp-imagemin', 'del', 'babel-register', 'gulp-cache', 'gulp-if', 'gulp-less', 'main-bower-files', 'gulp-plumber', 'gulp-uglify', 'gulp-size', 'gulp-load-plugins'], 'release_date': '2016-10-23T14:21:58.976Z'}, '1.1.1': {'dependencies': ['gulp-useref', 'browser-sync', 'gulp-eslint', 'babel-preset-es2015', 'gulp-sourcemaps', 'babel-core', 'babel-preset-react', 'gulp-htmlmin', 'gulp', 'wiredep', 'run-sequence', 'gulp-babel', 'gulp-cssnano', 'gulp-autoprefixer', 'gulp-imagemin', 'del', 'babel-register', 'gulp-cache', 'gulp-if', 'gulp-less', 'main-bower-files', 'gulp-plumber', 'gulp-uglify', 'gulp-size', 'gulp-load-plugins'], 'release_date': '2016-10-24T02:12:17.111Z'}, '1.0.1': {'dependencies': ['browser-sync', 'gulp

In [96]:
# dict - {target package : [(dependent package, release)]}
from datetime import datetime as dt
delrl = {}
adrl = {}
all_del_deps =[]
for k in dependent1.keys():
    adrl[k] = [] #list of all versions with dependency where previous version did not have it
    delrl[k] = [] #list of all version without dependency where previous version did have it
    for l in dependent1[k]:
        d = list(l.keys())[0]
        v = l[d]
        rdt_l = {}


        for rk in v.keys():
            r = v[rk]

            rdt = r['release_date'][:10]+' '+r['release_date'][11:19]
            rdt = dt.strptime(rdt , "%Y-%m-%d %H:%M:%S")
            try:
                if k in r['dependencies']: rdt_l[rdt] = ['Y',rk]
                else: rdt_l[rdt] = ['N', rk]
            except:
                continue

        rdt_s = sorted(rdt_l.keys())
        fadd = 0

        for r in rdt_s:
            if fadd == 0 and rdt_l[r][0] == 'Y':
                if r.date() < dt(2015,3,1).date(): t = dt(2015,3,1).date()
                else: t = r.date()
                adrl[k].append([d,  rdt_l[r][1], str(r)])
                fadd = 1
            if fadd and rdt_l[r][0] == 'N':
                if r.date() < dt(2015,3,1).date(): t = dt(2015,3,1).date()
                else: t = r.date()
                delrl[k].append([d, rdt_l[r][1], str(r)])
                all_del_deps.append(d)
                fadd = 0

In [109]:
for k in adrl:
    for r in range(0, len(adrl[k]) -1 ):
        print(adrl[k][r])

['jsonscript-proxy', '0.1.0', '2016-06-11 21:10:29']
['senasaikou-yeoman', '1.0.0', '2016-10-23 12:28:02']
['webpack-egoist', '2.2.0-rc.2', '2016-12-23 15:05:08']
['alex-d.js', '1.0.0', '2017-03-23 00:37:23']
['advanced-image-loader', '1.0.0', '2017-04-02 19:13:04']
['webpack-addons-ylvis', '0.0.27', '2017-02-05 14:01:06']
['primeng-custom', '4.0.0-beta.1', '2017-04-06 09:34:45']
['jicli', '0.0.1', '2017-03-30 01:47:09']
['@lgeiger/electron-builder', '1.0.0', '2017-04-09 21:24:46']
['@touched/map-tool-project', '0.1.0', '2017-04-23 15:24:49']
['generator-arc', '0.0.1', '2016-10-14 13:11:12']
['node-private-tools', '2.0.0', '2017-05-09 04:38:12']
['fhir2', '1.0.0', '2017-05-23 20:29:06']
['webpack-ie8', '2.1.0-beta.26', '2016-11-14 05:38:31']
['mocoolka-tools', '0.5.22', '2017-04-25 19:39:45']
['misstep', '1.0.0', '2017-06-14 19:34:09']
['@cysonius/schema-utils', '0.0.14', '2017-05-06 16:41:19']
['ng2-dnd-kokatsuna', '1.0.0', '2017-06-17 16:54:40']
['qweb-client', '1.0.0', '2017-06-27 1